In [ ]:
from multiprocessing import freeze_support
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
import torch

In [ ]:
tomatoes = load_dataset("rotten_tomatoes")
test_df = pd.DataFrame(tomatoes["test"])

In [ ]:
def embdding_and_consine_eval(test_df):
    model = SentenceTransformer('all-mpnet-base-v2')
    eval_embeddings = model.encode(test_df.text, show_progress_bar=True)
    label_embeddings = model.encode(["A very negative review", "A very positive review"])
    sim_matrix = cosine_similarity(eval_embeddings, label_embeddings)
    y_pred = np.argmax(sim_matrix, axis=1)
    print('Embedding and cosine eval')
    print(classification_report(test_df.label, y_pred))

embdding_and_consine_eval(test_df)

In [ ]:
def nli_eval(test_df):
    pipe = pipeline(model="facebook/bart-large-mnli", device=torch.device("mps"))
    candidate_labels_dict = {"negative movie review": 0, "positive movie review": 1}
    candidate_labels = ["negative movie review", "positive movie review"]
    predictions = pipe(test_df.text.tolist(), candidate_labels, multi_label=True)
    y_pred = [candidate_labels_dict[prediction["labels"][0]] for prediction in predictions]
    print('NLI eval')
    print(classification_report(test_df.label, y_pred))

nli_eval(test_df)